In [1]:
# def load_data(loc:str, index:list):
#     df = pandas.read_csv(f'data/{loc}_solar19.csv')
#     df['idx'] = index
#     df = df.set_index('idx')
#     return df
# idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]
# weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
# weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
# weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
# weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]""
# demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
# demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
# demand_ny = DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga
# ng_price = make_henry_price_df(
#     file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
# ng_price = ng_price.drop(columns= 'scales')
# ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
# ng_price = pandas.concat([ng_price])

In [2]:
import sys

sys.path.append('src')

In [3]:
from pandas import DataFrame
from energiapy.components import *
from energiapy.blocks.scenario import Scenario
from energiapy.parameters.designators.mode import X
from energiapy.parameters.designators.incidental import I

In [4]:
from energiapy.components._base._scope import _Scope
from energiapy.components._base._defined import _Defined
from energiapy.components.temporal.scale import Scale
from energiapy.components.spatial.location import Location
from sympy import pprint, pretty, latex

# for i in [
#     Player,
#     Cash,
#     Land,
#     Material,
#     Resource,
#     Emission,
#     Process,
#     Storage,
#     Transit,
#     Horizon,
#     Network,
#     Linkage,
#     Location,
#     Scale,
# ]:
#     # print(i.__name__, issubclass(i, _Scope))
#     print(i.__name__, issubclass(i, _Defined))

In [5]:
from dataclasses import dataclass, fields

In [6]:
from energiapy._core._aliases._is_input import IsInput

In [7]:
s = Scenario(def_emissions=True)

In [8]:
# s.h2 = Horizon({'days': 2, 'hours': 12})
s.hor = Horizon(discretizations=[2, 12])
s.scales
s.net = Network(['madgaon', 'ponje', 'cacoda'])  # , link_all=True)
s.nodes
s.alink = Linkage(source=s.madgaon, sink=s.ponje, bi=True, distance=50)
s.blink = Linkage(source=s.ponje, sink=s.cacoda, bi=False, distance=80)
s.clink = Linkage(source=s.cacoda, sink=s.madgaon, bi=True, distance=100)
s.dlink = Linkage(source=s.madgaon, sink=s.ponje, bi=False, distance=200)

In [9]:
s.alink.sib

alink_

In [10]:
s.alink_.sib

alink

In [11]:
s.eqns()

<generator object Scenario.eqns at 0x11e7f50e0>

In [12]:
for l in s.linkages:
    print(l.source, l.sink, l.sib, l.distance)

madgaon ponje alink_ 50
ponje madgaon alink 50
ponje cacoda None 80
cacoda madgaon clink_ 100
madgaon cacoda clink 100
madgaon ponje None 200


In [13]:
s.datasets

[]

In [14]:
a = DataFrame({'a': list(range(2))})
b = DataFrame({'b': list(range(24))})

In [15]:
s.csh = Cash(
    spend={
        s.madgaon: True,
        s.ponje: {s.t1: (2, a)},
        s.cacoda: 300,
        s.network: {s.t2: [5, 60]},
    },
    label='cash',
    basis='USD',
)

dp [5, 60]
2
v [M[csh, madgaon, t0], 300, θ[csh, ponje, t1], [5, 60]]


In [16]:
s.csh.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
s.csh.constraints

[Bind[csh, cacoda, t0],
 Bind[csh, madgaon, t0],
 Bind[csh, net, t2]max,
 Bind[csh, net, t2]min,
 Bind[csh, ponje, t1]]

In [17]:
s.csh.eqns()

<generator object _Defined.eqns at 0x00000245FDC9E5E0>

In [18]:
s.csh.parameters

[Spend[csh, cacoda, t0],
 Spend[csh, madgaon, t0],
 Spend[csh, net, t2],
 Spend[csh, ponje, t1]]

In [19]:
s.data.__dict__

{'name': 'Data|:s:|',
 '_initial_state': {'name': ':s:'},
 'gwp': Data|gwp|,
 'ap': Data|ap|,
 'epm': Data|epm|,
 'epf': Data|epf|,
 'ept': Data|ept|,
 'pocp': Data|pocp|,
 'odp': Data|odp|,
 'adpmn': Data|adpmn|,
 'adpmt': Data|adpmt|,
 'adpf': Data|adpf|,
 'wdp': Data|wdp|,
 'csh': Data|csh|}

In [20]:
s.csh.data.__dict__

{'component': csh,
 'name': 'Data|csh|',
 'spend': [M[csh, madgaon, t0], 300, θ[csh, ponje, t1], [5, 60]]}

In [21]:
s.parameters

[Spend[csh, cacoda, t0],
 Spend[csh, madgaon, t0],
 Spend[csh, net, t2],
 Spend[csh, ponje, t1]]

In [22]:
s.csh.parameters

[Spend[csh, cacoda, t0],
 Spend[csh, madgaon, t0],
 Spend[csh, net, t2],
 Spend[csh, ponje, t1]]

In [23]:
s.lnd = Land(use={s.t2: 100, s.ponje: {s.t0: b}, s.cacoda: a}, cost=20)

dp DtSt[lnd, cacoda, t1]
2
v [DtSt[lnd, cacoda, t1], 100, DtSt[lnd, ponje, t2]]
dp 20
1
v [20]


c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:164: UserWarning: lnd.use:Inconsistent temporal scale for ponje at t0. Updating to t2
  warn(


In [24]:
# s.lnd.parameters
# s.lnd.constraints
# s.lnd.variables

In [25]:
s.Ra = Resource(buy_emission={s.gwp: {s.t2: b}})

dp DtSt[gwp, Ra, net, t2]
24
v [DtSt[gwp, Ra, net, t2]]


In [26]:
s.Ra.eqns()

<generator object _Defined.eqns at 0x00000245FDD39230>

In [17]:
s.H2 = Resource(
    sell={s.t0: [0, 4], s.t1: [a, 34], s.t2: (a, b)},
    sell_price={
        s.madgaon: True,
        s.ponje: {s.t1: (2, b)},
        s.cacoda: 300,
        s.net: {s.t0: [5, 60]},
    },
)


s.Solar = Resource(buy=True, basis='MW', label='Solar Power')


s.Wind = Resource(buy=a, basis='MW', label='Wind Power')


s.Power = Resource(basis='MW', label='Power generated')


s.Uranium = Resource(
    buy=DataFrame({'a': [i for i in range(24)]}),
    buy_price=42.70 / (250 / 2),
    basis='kg',
    label='Uranium',
)


s.H2_L = Resource(sell=(0, 23), basis='tons', label='Hydrogen')


s.CO2_AQoff = Resource(basis='tons', label='Carbon dioxide - sequestered')


s.H2O = Resource(buy=(20, 50), buy_price=b, basis='tons', label='Water')


s.CH4 = Resource(buy=[20, 40], buy_price=20, basis='tons', label='Natural gas')


s.CO2 = Resource(basis='tons', label='Carbon dioxide', block='Resource')


s.CO2_Vent = Resource(basis='tons', label='Carbon dioxide - Vented', sell_price=(2, 20))


s.O2 = Resource(sell=[20, True], basis='tons', label='Oxygen')


s.CO2_DAC = Resource(basis='tons', label='Carbon dioxide - captured')
s.Power = Resource(buy=[0, a], basis='MW', label='Power generated')

dp θ[H2, net, t2]
24
v [[0, 4], [DtSt[H2, net, t1], 34], θ[H2, net, t2]]
dp [5, 60]
2
v [M[csh, H2, madgaon, t0], 300, [5, 60], θ[csh, H2, ponje, t2]]
dp M[Solar, net, t0]
1
v [M[Solar, net, t0]]
dp DtSt[Wind, net, t1]
2
v [DtSt[Wind, net, t1]]
dp DtSt[Uranium, net, t2]
24
v [DtSt[Uranium, net, t2]]
dp 0.3416
1
v [0.3416]
dp θ[H2_L, net, t0]
1
v [θ[H2_L, net, t0]]
dp θ[H2O, net, t0]
1
v [θ[H2O, net, t0]]
dp DtSt[csh, H2O, net, t2]
24
v [DtSt[csh, H2O, net, t2]]
dp [20, 40]
2
v [[20, 40]]
dp 20
1
v [20]
dp θ[csh, CO2_Vent, net, t0]
1
v [θ[csh, CO2_Vent, net, t0]]
dp [20, M[O2, net, t0]]
2
v [[20, M[O2, net, t0]]]
dp [0, DtSt[Power, net, t1]]
2
v [[0, DtSt[Power, net, t1]]]


/Users/cacodcar/Desktop/repos/energiapy/src/energiapy/components/_base/_consistent.py:202: UserWarning: H2.sell_price:Inconsistent temporal scale for ponje at t1. Updating to t2
  warn(
/Users/cacodcar/Desktop/repos/energiapy/src/energiapy/blocks/_base/_ok.py:49: UserWarning: Power already exists, overwriting
set Scenario.ok_overwrite=False, to enforce strictly
  warn(
/Users/cacodcar/Desktop/repos/energiapy/src/energiapy/components/_base/_consistent.py:202: UserWarning: Power.buy:Inconsistent temporal scale for net at t0. Updating to t1
  warn(


In [28]:
s.O2.sell

[[20, M[O2, net, t0]]]

In [29]:
from sympy import latex

latex(s.H2.constraints[2].equation)

'{sell}_{H_{2},net,t_{1}} \\leq 34'

In [20]:
s.latex()

'{spend}_{csh,cacoda,t_{0}} = 300'

'{spend}_{csh,madgaon,t_{0}} = {M}_{csh,madgaon,t_{0}}'

'{spend}_{csh,net,t_{2}} \\leq 60'

'{spend}_{csh,net,t_{2}} \\geq 5'

'{spend}_{csh,ponje,t_{1}} = {θ}_{csh,ponje,t_{1}}'

...

'{buy}_{CH_{4},net,t_{0}} \\geq 20'

'{exp^{buy}}_{csh,CH_{4},net,t_{0}} = {Price^{buy}}_{csh,CH_{4},net,t_{0}} \\cdot {buy}_{CH_{4},net,t_{0}}'

'{exp^{sell}}_{csh,CO_{2 Vent},net,t_{0}} = {Price^{sell}}_{csh,CO_{2 Vent},net,t_{0}} \\cdot {sell}_{CO_{2 Vent},net,t_{0}}'

'{sell}_{O_{2},net,t_{0}} = {M}_{O_{2},net,t_{0}}'

'{sell}_{O_{2},net,t_{0}} \\geq 20'

In [19]:
s.H2.latex()

'{sell}_{H_{2},net,t_{0}} \\leq 4'

'{sell}_{H_{2},net,t_{0}} \\geq 0'

'{sell}_{H_{2},net,t_{1}} \\leq 34'

'{sell}_{H_{2},net,t_{1}} \\geq {DtSt}_{H_{2},net,t_{1}}'

'{sell}_{H_{2},net,t_{2}} = {θ}_{H_{2},net,t_{2}}'

'{exp^{sell}}_{csh,H_{2},cacoda,t_{0}} = {Price^{sell}}_{csh,H_{2},cacoda,t_{0}} \\cdot {sell}_{H_{2},cacoda,t_{0}}'

'{exp^{sell}}_{csh,H_{2},madgaon,t_{0}} = {Price^{sell}}_{csh,H_{2},madgaon,t_{0}} \\cdot {sell}_{H_{2},madgaon,t_{0}}'

'{exp^{sell}}_{csh,H_{2},net,t_{0}} = {Price^{sell}}_{csh,H_{2},net,t_{0}} \\cdot {sell}_{H_{2},net,t_{0}}'

'{exp^{sell}}_{csh,H_{2},ponje,t_{2}} = {Price^{sell}}_{csh,H_{2},ponje,t_{2}} \\cdot {sell}_{H_{2},ponje,t_{2}}'

In [30]:
s.LiR = Material(
    emission={s.gwp: 1.484},
    use={s.ponje: 1.5},
    basis='kg',
    label='Lithium Reserves',
    citation='Nelson Bunyui Manjong (2021)',
)


# LiB = Material(name='LiB', gwp=0.031, toxicity=793, basis='kg', label='Brine-based Lithium',


#                citation='Nelson Bunyui Manjong (2021)')


# Monocrystalline = Material(name='Mc', gwp=20, resource_cons={H2O: 1.5}, toxicity=40.74, basis='kg', label='Mono',


#                            citation='Environmental impact assessment of monocrystalline silicon solar photovoltaic cell production: a case study in China (toxicity)')


# Polycrystalline = Material(name='Pc', gwp=26, resource_cons={


#                            H2O: 1}, toxicity=15, basis='kg', label='Poly', citation='Environmental Economic Impact Assessments of a ... - MDPI (gwp and toxicity)')


# CdTE = Material(name='CdTe', gwp=40, resource_cons={H2O: 0.25}, toxicity=5, basis='kg', label='C',


#                 citation='Investigation of life cycle CO2 emissions of the polycrystalline and cadmium telluride PV panels')

dp 1.5
1
v [1.5]
dp 1.484
1
v [1.484]


In [31]:
s.WF = Process(
    conversion={s.Power: {X(0): {s.Wind: -1}, X(1): {s.Wind: -1}}},
    produce=[True, b],
    capex={s.madgaon: {s.t1: {1462000, I(202233)}}},
    opex={4953, I(70)},
    capacity=1000,
    emission={s.gwp: 50, s.odp: 2900},
    material={s.LiR: 1.5},
    locations=[s.madgaon, s.ponje],
    land=30,
    label='Wind mill array',
    basis='MW',
)


s.PV = Process(
    sell_price={s.Power: 30},
    credit={s.Power: DataFrame({'a': [2]})},
    conversion={s.Power: {X(1): {s.Solar: -1.2}, X(2): {s.Solar: -1.5}}},
    capex={X(0): 1333, X(1): 1444, X(2): 1555},
    opex=22623,
    capacity={X(0): 3000, X(1): 4000, X(2): 5000},
    locations=s.cacoda,
    label='Solar PV',
    basis='MW',
)


s.SMRH = Process(
    land=50,
    conversion={
        s.H2: {
            s.Power: -1.11,
            s.CH4: -3.76,
            s.H2O: -23.7,
            s.CO2_Vent: 1.03,
            s.CO2: 9.332,
        }
    },
    capex={
        2520000,
    },
    opex={51.5, I(945000)},
    capacity=1000,
    label='Steam methane reforming + CCUS',
)

s.NGCC = Process(
    buy_price={s.CH4: 4, s.H2O: 20},
    conversion={
        s.Power: {
            s.CH4: -0.108,
            s.H2O: -10,
            s.CO2_Vent: 0.297 * 0.05,
            s.CO2: 0.297 * 0.95,
        }
    },
    capex=2158928,
    opex={4090, I(53320)},
    capacity=[1, 100],
    label='NGCC + 95% CC',
)
s.SMR = Process(
    capex=2400,
    opex={0.03, I(800)},
    conversion={s.H2: {s.Power: -1.11, s.CH4: -3.76, s.H2O: -23.7, s.CO2_Vent: 9.4979}},
    capacity=1000,
    label='Steam methane reforming',
)
s.H2FC = Process(
    buy_price={s.H2: 2},
    sell={s.Power: 20},
    conversion={s.Power: {s.H2: -0.050}},
    capex=1.6 * 10**6,
    opex=3.5,
    capacity=[1000, 2000],
    label='hydrogen fuel cell',
)
s.DAC = Process(
    capex=0.02536,
    opex=0.634,
    conversion={s.CO2_DAC: {s.Power: -0.193, s.H2O: -4.048}},
    capacity=1000,
    label='Direct air capture',
)
# s.PSH = Process(conversion = {s.Power: 0.6}, capex = 3924781, fopex= 17820, vopex = 512.5, store = 10000, capacity=1000, label='Pumped storage hydropower', basis = 'MW')
s.ASMR = Process(
    buy={s.Uranium: 40},
    conversion={s.Power: {s.Uranium: -4.17 * 10 ** (-5), s.H2O: -3.364}},
    capex=7988951,
    opex=I(0.04 * 0.730),
    capacity=1000,
    label='Small modular reactors (SMRs)',
)
s.AWE = Process(
    land=(0, 20),
    conversion={s.H2: {s.Power: -1, s.O2: 0.7632, s.H2O: -0.1753}},
    capex={1.1 * 10**6, I(20)},
    opex=I(16918),
    capacity=True,
    label='Alkaline water electrolysis (AWE)',
    citation='Demirhan et al. 2018 AIChE paper',
)

c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:202: UserWarning: WF.operate:Inconsistent temporal scale for net at t0. Updating to t2
  warn(


dp 1000
1
v [1000]
dp [M[WF, net, t2], DtSt[WF, net, t2]]
2
v [[M[WF, net, t2], DtSt[WF, net, t2]]]
dp {202233, 1462000}
2
v [{202233, 1462000}]
dp {4953, 70}
2
v [{4953, 70}]
dp 2900
1
v [50, 2900]
dp 30
1
v [30]
dp 1.5
1
v [1.5]
dp 5000
1
v [3000, 4000, 5000]
dp [0, 1]
2
v [[0, 1]]
dp 1555
1
v [1333, 1444, 1555]
dp 22623
1
v [22623]
dp 30
1
v [30]
dp DtSt[csh, Power, PV, net, t0]
1
v [DtSt[csh, Power, PV, net, t0]]
dp 1000
1
v [1000]
dp [0, 1]
2
v [[0, 1]]
dp {2520000}
1
v [{2520000}]
dp {945000, 51.5}
2
v [{945000, 51.5}]
dp 50
1
v [50]
dp [1, 100]
2
v [[1, 100]]
dp [0, 1]
2
v [[0, 1]]
dp 2158928
1
v [2158928]
dp {53320, 4090}
2
v [{53320, 4090}]
dp 20
1
v [4, 20]
dp 1000
1
v [1000]
dp [0, 1]
2
v [[0, 1]]
dp 2400
1
v [2400]
dp {800, 0.03}
2
v [{800, 0.03}]
dp [1000, 2000]
2
v [[1000, 2000]]
dp [0, 1]
2
v [[0, 1]]
dp 1600000.0
1
v [1600000.0]
dp 3.5
1
v [3.5]
dp 20
1
v [20]
dp 2
1
v [2]
dp 1000
1
v [1000]
dp [0, 1]
2
v [[0, 1]]
dp 0.02536
1
v [0.02536]
dp 0.634
1
v [0.634]
dp 1000
1


In [31]:
list(s.WF.conversion.balance)[0]

x_0_WF

In [36]:
type(list(s.WF.capex[0])[0])

energiapy.parameters.values.constant.Constant

In [33]:
s.PV.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [33]:
from sympy import symbols

list(s.WF.conversion.balance)

[x_0_WF, x_1_WF]

In [35]:
type(s.WF.conversion)

energiapy.parameters.balance.conversion.Conversion

In [33]:
s.WF.parameters[0]

CapF[WF, net, t2]

In [36]:
s.WF.parameters[0].disposition.args()

{'ply': None,
 'emn': None,
 'csh': None,
 'res': None,
 'mat': None,
 'lnd': None,
 'pro': WF,
 'stg': None,
 'trn': None,
 'loc': None,
 'lnk': None,
 'ntw': net,
 'scl': t2,
 'mde': None}

In [35]:
list(s.WF.conversion.balance)[0].name

'(x(WF, conv, 0))'

In [35]:
s.WF.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
s.LiI = Storage(capacity=[2000], inventory=s.Power, capacity_c=[40], capacity_d=[20])
s.LiI2 = Storage(capacity=[2000], inventory={s.Power: 0.6})
s.H2Stg = Storage(
    capacity=[2000], inventory={s.H2: {X(0): {s.Power: 0.8}, X(1): {s.Power: 0.9}}}
)

In [ ]:
s.resources

[CH4,
 CO2,
 CO2_AQoff,
 CO2_DAC,
 CO2_Vent,
 H2,
 H2O,
 H2Stg_H2,
 H2_L,
 LiI2_Power,
 LiI_Power,
 O2,
 Power,
 Ra,
 Solar,
 Uranium,
 Wind]

In [ ]:
s.LiI2.conversion_c.balance

{LiI2_Power: 1, Power: -1.6666666666666667}

In [ ]:
s.LiI

LiI

In [ ]:
s.LiI_d

LiI_d

In [ ]:
s.AWE.parameters

[CapF[AWE, net, t0],
 Capacity[AWE, net, t0],
 Capex[csh, AWE, net, t0],
 Capex^i[csh, AWE, net, t0],
 Land[lnd, AWE, net, t0],
 Mat[lnd, AWE, net, t0],
 Opex^i[csh, AWE, net, t0]]

In [ ]:
s.LiI_c.latex()

'{capacity}_{LiI_{c},net,t_{0}} \\leq 40'

'{capacity}_{LiI_{c},net,t_{0}} \\geq 0'

In [ ]:
s.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [40]:
s.parameters

[Spend[csh, cacoda, t0],
 Spend[csh, madgaon, t0],
 Spend[csh, net, t2],
 Spend[csh, ponje, t1],
 Cost^use[csh, lnd, net, t0],
 Use[lnd, cacoda, t1],
 Use[lnd, net, t2],
 Use[lnd, ponje, t2],
 Emit^buy[gwp, Ra, net, t2],
 Price^sell[csh, H2, cacoda, t0],
 Price^sell[csh, H2, madgaon, t0],
 Price^sell[csh, H2, net, t0],
 Price^sell[csh, H2, ponje, t2],
 Sell[H2, net, t0],
 Sell[H2, net, t1],
 Sell[H2, net, t2],
 Buy[Solar, net, t0],
 Buy[Wind, net, t1],
 Buy[Power, net, t1],
 Buy[Uranium, net, t2],
 Price^buy[csh, Uranium, net, t0],
 Sell[H2_L, net, t0],
 Buy[H2O, net, t0],
 Price^buy[csh, H2O, net, t2],
 Buy[CH4, net, t0],
 Price^buy[csh, CH4, net, t0],
 Price^sell[csh, CO2_Vent, net, t0],
 Sell[O2, net, t0],
 Emit^use[gwp, LiR, net, t0],
 Use[LiR, ponje, t0],
 CapF[WF, net, t2],
 Capacity[WF, net, t0],
 Capex[csh, WF, madgaon, t1],
 Capex^i[csh, WF, madgaon, t1],
 Emit^opn[gwp, WF, net, t0],
 Emit^opn[odp, WF, net, t0],
 Land[LiR, WF, net, t0],
 Land[lnd, WF, net, t0],
 Mat[LiR, WF, n

In [ ]:
s.AWE.eqns()

In [ ]:
s.datasets

In [ ]:
s.train = Transit(
    carries=s.H2,
    ship=[300],
    capacity=[400],
    land=20,
    emission={s.odp: 0.1},
    capex={s.alink: 200, s.blink: 300, s.clink: 400},
    opex=100,
    label='Train',
    basis='passengers',
    linkages=[s.alink],
)

In [ ]:
s.transits

In [ ]:
s.train.linkages

In [ ]:
s.train.eqns()

In [ ]:
s.Wind.locations

In [ ]:
s.LiI_c.__dict__

In [ ]:
s.LiI_c.eqns()

In [ ]:
s.processes

In [ ]:
for i in s.resources:
    print(i, i.is_located)

In [ ]:
s.WF.locations

In [ ]:
s.AWE.locations

In [ ]:
s.PV.conversion

In [ ]:
s.eqns()

In [ ]:
from collections import OrderedDict

In [ ]:
a = OrderedDict((i, 't') for i in [(1, 2), (3, 4), (5, 6)])

In [ ]:
s.dispositions

In [ ]:
a[(1, 2)]

In [ ]:
s.WF.locations

In [ ]:
s.madgaon.processes

In [ ]:
s.cacoda.processes

In [ ]:
s.WF.eqns()

In [ ]:
s.PV._spatials()

In [ ]:
s.PV.conversion.involved

In [ ]:
s.cacoda.resources

In [ ]:
s.network.processes

In [ ]:
s.cacoda.materials

In [ ]:
s.ponje.materials

In [ ]:
s.ponje.emissions

In [ ]:
s.WF.emission[0].disposition.emn

In [ ]:
s.ponje.commodities

In [ ]:
s.H2

In [ ]:
[loc for loc in s.H2.system.locations if s.H2 in loc.commodities]

In [ ]:
s.H2.locations

In [ ]:
s.H2.linkages

In [ ]:
s.alink.commodities

In [ ]:
s.ponje.commodities

In [ ]:
s.Wind.locations

In [ ]:
s.cacoda.resources

In [ ]:
s.LiR.locations

In [ ]:
s.ponje.resources

In [ ]:
Process.inputs()

In [ ]:
Storage.inputs()

In [ ]:
s.LiI_c

In [ ]:
s.H2Stg.eqns()

In [ ]:
s.H2Stg.balance_c

In [ ]:
s.LiI.balance_d

In [ ]:
# s.H2Stg.cin.balance
s.H2Stg_c.conversion.balance

In [ ]:
s.LiI.eqns()

In [ ]:
s.LiI_c.eqns()

In [ ]:
s.LiI2_d.conversion.balance
s.H2Stg_d.conversion.balance

In [ ]:
s.resources

In [ ]:
isinstance(s.LiI_Power, Resource)

In [ ]:
help(Resource

In [ ]:
from energiapy.components._base._component import _Component
from energiapy.components._base._defined import _Defined

In [ ]:
isinstance(s.H2, _Defined)

In [ ]:
s.storages

In [ ]:
s.processes

In [ ]:
s.LiI_c.conversion.__dict__

In [ ]:
s.WF.operate

In [ ]:
s.data.__dict__

In [ ]:
s.SMR.constants

In [ ]:
s.SMR.eqns()

In [ ]:
s.eqns()

In [ ]:
s.WF.dispositions

In [ ]:
a = [1].insert(0, 0)

In [ ]:
a

In [ ]:
s.components

In [ ]:
s.WF.data

In [ ]:
s.WF.constraints[0].__dict__

In [ ]:
s.constants

In [ ]:
from energiapy._core._aliases._is_component import IsComponent

In [ ]:
from enum import Enum, auto

In [ ]:
class A(Enum):
    a = auto()
    b = auto()

In [ ]:
@dataclass
class B:
    a: A

In [ ]:
q = B(A.a)
r = B(A.b)
z = B(A.a)
w = B(A.b)
l = [q, r, z, w]

In [ ]:
l_a = [i for i in l if i.a == A.a]

In [ ]:
l_a

In [ ]:
s.PV.parameters

In [ ]:
s.PV.constraints

In [ ]:
s.PV.variables[4].disposition.__dict__

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class Ra:
    a: int = 0

    @classmethod
    def b(self, b):
        return self.a + b

In [ ]:
Ra.b(2)

In [ ]:
s.PV.capex[0].disposition.__dict__

In [ ]:
s.WF.variables

In [ ]:
s.WF.parameters

In [ ]:
s.WF._consistent

In [ ]:
s.WF.__dict__

In [ ]:
s.PV.variables

In [ ]:
s.variables

## Declare processes

In [ ]:
# s.LiI = Process(store= 2000, conversion = {s.Power: 0.9}, store_loss = 0.2, store_cost= 0.4, capex = 3516428, fopex= 87910, vopex = 0, capacity= [1000,BigM], label='Lithium-ion battery', basis = 'MW')

## Declare locations

In [ ]:
PV.cap_max

In [ ]:
PV.factors

In [ ]:
houston.aspect

In [ ]:
houston.land_cost_factor

In [ ]:
# for i in houston.processes | sandiego.processes | newyork.processes:
# # for i in houston.resources | sandiego.resources | newyork.resources:
#     print(i, i.localizations)
#     print(i, i.factors)

In [ ]:
Power.ctype

## Declare transports

In [ ]:
# Train_H2 = Transport(name='Train_H2', resources={
#                      H2}, trans_max=bigM, trans_loss=0.03, trans_cost=1.667*10**(-3), label='Railway for hydrogen transportation')
# Grid = Transport(name='Grid', resources={
#                  Power}, trans_max=bigM, trans_loss=0.001, trans_cost=0.5*10**(-3), label='Railroad transport')


Train_H2 = Transport(
    name='Train_H2',
    resources={H2},
    cap_max=BigM,
    trans_loss=0.03,
    capex=1000,
    vopex=1.667,
    fopex=(0, 1),
    acid=34,
    eutf=55,
    label='Railway for hydrogen transportation',
)
Grid = Transport(
    name='Grid',
    resources={H2, Power},
    cap_max=BigM,
    trans_loss=0.001,
    capex=500,
    vopex=0.444,
    label='Railroad transport',
)

In [ ]:
Train_H2.cap_max

## Declare Network

In [ ]:
distance_matrix = [[2366], [2620]]

transport_matrix = [
    [[Train_H2]],
    [[Grid]],
]
capacity_factor = {
    (houston, newyork): {Train_H2: weather_sd[['wind_speed']]},
    (sandiego, newyork): {Train_H2: weather_sd[['wind_speed']]},
}

network = Network(
    name='network',
    scales=scales,
    land_max=(0, 5),
    land_max_factor=DataFrame({'a': [5]}),
    capacity_factor=capacity_factor,
    vopex_factor={(houston, sandiego): {Train_H2: weather_sd[['wind_speed']]}},
    sources=[houston, sandiego],
    sinks=[newyork],
    distance_matrix=distance_matrix,
    transport_matrix=transport_matrix,
)

In [ ]:
scenario = Scenario(
    name='scenario_full', network=network, scales=scales, label='full_case'
)

In [ ]:
scenario

In [ ]:
for i in houston.resources:
    print(i.ctype)

In [ ]:
houston.resources_produce

In [ ]:
ResourceType.all()

In [ ]:
ResourceType.resource_level()

In [ ]:
ResourceType.location_level()

In [ ]:
ResourceType.transport_level()

In [ ]:
ProcessType.all()

In [ ]:
ProcessType.process_level()

In [ ]:
ProcessType.location_level()

In [ ]:
LocationType.location_level()

In [ ]:
Power.transport

In [ ]:
Solar.availability

In [ ]:
Power.demand

In [ ]:
[((houston, sandiego), CO2), ((houston, sandiego), H2), ((houston, newyork), CO2)]

In [ ]:
comp_loc_dict = scenario.resources_locations_discharge

In [ ]:
scenario.locations_land

In [ ]:
scenario.locations_locations_land

In [ ]:
comp_loc_ordered_set = [(i, j) for i in comp_loc_dict for j in comp_loc_dict[i]]

In [ ]:
scenario.resources_locations_store

In [ ]:
comp_loc_ordered_set

In [ ]:
comp_loc_ordered_set

In [ ]:
scenario.resources_locations_discharge

In [ ]:
scenario.resources_discharge

In [ ]:
scenario.processes_locations_expenditure

In [ ]:
resources_consume_locations = {
    resource: set() for resource in scenario.resources_consume
}
for resource in scenario.resources_consume:
    for location in scenario.locations:
        if hasattr(location, 'resources_consume'):
            if resource in location.resources_consume:
                resources_consume_locations[resource].add(location)

In [ ]:
resources_consume_locations = {
    resource: {
        location
        for location in scenario.locations
        if hasattr(location, 'resources_consume')
        and resource in location.resources_consume
    }
    for resource in scenario.resources_consume
}

In [ ]:
scenario.resources_consume

In [ ]:
resources_consume_locations = {
    resource: {
        location
        for location in self.locations
        if hasattr(location, 'resources_consume')
        and resource in location.resources_consume
    }
    for resource in self.resources_consume
}

In [ ]:
resources_consume_locations

In [ ]:
resources_demand_locations = {
    resource: {
        location
        for location in scenario.locations
        if hasattr(location, 'resources_demand')
        and resource in location.resources_demand
    }
    for resource in scenario.resources_demand
}

In [ ]:
scenario.resource_classifications()

In [ ]:
{
    component: {
        location
        for location in getattr(self, 'locations')
        if hasattr(location, f'{component_set}_{parameter.lower()}')
        and resource in getattr(location, f'{component_set}_{parameter.lower()}')
    }
    for resource in getattr(self, f'{component_set}_{parameter.lower()}')
}

In [ ]:
scenario.process_classifications()

In [ ]:
[
    ((houston.name, sandiego.name), CO2.name),
    ((houston.name, sandiego.name), H2.name),
    ((houston.name, newyork.name), CO2.name),
]

In [ ]:
help(ResourceParamType)

In [ ]:
from pyomo.environ import *

# from energiapy.utils.scale_utils import scale_pyomo_set

In [ ]:
scales1 = TemporalScale(discretization_list=[1, 4], design_scale=0, scheduling_scale=1)

In [ ]:
type(scales1)

In [ ]:
scales2 = TemporalScale(
    discretization_list=[2, 2, 2], design_scale=0, scheduling_scale=2
)

In [ ]:
scales1.index_list

In [ ]:
scales2.index_list

In [ ]:
m = ConcreteModel()
m.scales = Set(scales1.list, initialize=scales1.scale)
m.scales_scheduling = scale_pyomo_set(instance=m, scale_level=scales1.scheduling_scale)
m.scales_design = scale_pyomo_set(instance=m, scale_level=scales1.design_scale)

In [ ]:
scales.scheduling_scale

In [ ]:
# m.a = Set(initialize = [(i, j) for i in resources_consume_locations for j in resources_consume_locations[i]])
# m.b = Set(initialize = [(i.name, j.name) for i in scenario.resources_locations_consume for j in scenario.resources_locations_consume[i]])
# m.c = Set(initialize = [((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)])
# m.A = Var(m.a, m.scales_scheduling, within = NonNegativeReals)
# m.B = Var(m.b, m.scales_design, within = NonNegativeReals)
# m.C = Var(m.c, m.scales_scheduling, within = NonNegativeReals)

In [ ]:
from pyomo.environ import *

m = ConcreteModel()

In [ ]:
m.d = Set(
    initialize=[
        (Train_H2.name, (houston.name, sandiego.name)),
        (Grid.name, (houston.name, sandiego.name)),
        (Grid.name, (houston.name, newyork.name)),
    ]
)
m.D = Var(m.d, m.scales_design, within=NonNegativeReals)

In [ ]:
m.f = Set(initialize=[Train_H2.name, Grid.name])
m.F = Var(m.f, m.scales_design, within=NonNegativeReals)

In [ ]:
m.g = Set(
    initialize=[
        (CO2.name, (houston.name, sandiego.name)),
        (Power.name, (houston.name, sandiego.name)),
        (H2.name, (houston.name, newyork.name)),
    ]
)

In [ ]:
m.G = Var(m.g, m.scales_scheduling, within=NonNegativeReals)

In [ ]:
m.G.pprint()

In [ ]:
m = ConcreteModel()

In [ ]:
import gurobipy

m = gurobipy.read('example.lp')

In [ ]:
import pandas

In [ ]:
a = DataFrame(data={'a': [1, 2, 3]})

In [ ]:
b = 

In [ ]:
class LPFileCreator:
    def __init__(self, objective, constraints, bounds, file_name="problem.lp"):
        self.objective = objective
        self.constraints = constraints
        self.bounds = bounds
        self.file_name = file_name

    def create_lp_file(self):
        content = self._generate_lp_content()
        with open(self.file_name, "w") as file:
            file.write(content)

    def _generate_lp_content(self):
        lp_content = []

        # Add Objective Function
        lp_content.append("\\* Problem Statement *\\")
        lp_content.append("\nMaximize")
        lp_content.append(f" obj: {self.objective}\n")

        # Add Constraints
        lp_content.append("Subject To")
        for i, constraint in enumerate(self.constraints, start=1):
            lp_content.append(f" c{i}: {constraint}")
        lp_content.append("")

        # Add Bounds
        lp_content.append("Bounds")
        for bound in self.bounds:
            lp_content.append(f" {bound}")
        lp_content.append("")

        # Add End
        lp_content.append("End")

        return "\n".join(lp_content)

In [ ]:
a = [7, 2, 3].sort()

In [ ]:
a

In [ ]:
sorted(a)

In [ ]:
houston.resources_consume

In [ ]:
[f'cons_{i.name}_' for i in houston.resources_consume]

In [ ]:
def tuples_to_strings(tuple_list):
    return ['_'.join(map(str, t)) for t in tuple_list]

In [ ]:
scales = TemporalScale([1, 4], design_scale=0, scheduling_scale=1)

In [ ]:
var = ['cons']  # , 'sell']

In [ ]:
res_list = ['res1', 'res2', 'res3']

In [ ]:
time_index = tuples_to_strings(scales.index_list[1])

In [ ]:
value = {'res1': 5, 'res2': 7, 'res3': 8}

In [ ]:
cons = list()

In [ ]:
for i, j, k in product(var, res_list, time_index):
    cons.append(f'{i}_{j}_{k} <= {value[j]}')

In [ ]:
ooo = {'res1': 7, 'res2': 5, 'res3': 9}

In [ ]:
k = [f'{ooo[i]} {i}' for i in res_list]

In [ ]:
obj = ' + '.join(k)
obj

In [ ]:
for i,j,k in product(var, res_list, time_index):


In [ ]:
bou = [f'0 <= {i}' for i in var]

In [ ]:
# Define the problem components
objective = "x + 2 y"
constraints = ["x + y >= 1", "x <= 2", "y <= 2"]
bounds = ["0 <= x", "0 <= y"]

# Create the LP file
# creator = LPFileCreator(objective, constraints, bounds, file_name="example.lp")

creator = LPFileCreator(obj, cons, bou, file_name="example.lp")
creator.create_lp_file()
print("LP file 'example.lp' created successfully.")

In [ ]:
import gurobipy as gb

a = gb.read('example.lp')

In [ ]:
a.optimize()

In [ ]:
Maximize
  x + y + z
Subject To
  c0: x + y = 1
  c1: x + 5 y + 2 z <= 10
  qc0: x + y + [ x ^ 2 - 2 x * y + 3 y ^ 2 ] <= 5
Bounds
  0 <= x <= 5
  z >= 2
Generals
  x y z
End


In [ ]:
H2O.purchase_cost

In [ ]:
H2O.cons_max

In [ ]:
m.optimize()

In [ ]:
m.getVars()

In [ ]:
plot_scenario.capacity_factor(
    scenario=scenario, location=houston, process=PV, fig_size=(9, 5), color='orange'
)

![cap fac pv](plots / cf_ho.png)

In [ ]:
plot_scenario.capacity_factor(
    scenario=scenario, location=sandiego, process=WF, fig_size=(9, 5), color='blue'
)

![cap fac wf](plots / wf_sd.png)

In [ ]:
plot_scenario.demand_factor(
    scenario=scenario, location=newyork, resource=Power, fig_size=(9, 5), color='red'
)

![df ny](plots / df_ny.png)

## Optimize for minimum cost

In [ ]:
milp_cost = formulate(
    scenario=scenario,
    constraints={
        Constraints.COST,
        Constraints.INVENTORY,
        Constraints.PRODUCTION,
        Constraints.RESOURCE_BALANCE,
        Constraints.TRANSPORT,
        Constraints.MODE,
        Constraints.NETWORK,
    },
    objective=Objective.COST,
)

**Ensure no discharge of power in Houston and San Diego**

While the demand for power is zero in these locations, Power is still a dischargeable resource. Hence, the discharge bounds need to be set to zero.
If not set to zero, the system chooses to dispense power instead of establishing storage networks.

In [ ]:
milp_cost.constraint_specific_location_discharge_ho = (
    constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location=houston
    )
)
milp_cost.constraint_specific_location_discharge_sd = (
    constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location=sandiego
    )
)

In [ ]:
results_cost = solve(
    scenario=scenario,
    instance=milp_cost,
    solver='gurobi',
    name="results_cost",
    print_solversteps=True,
)

**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(
    results=results_cost, source='SD', sink='NY', resource='H2', transport='Train_H2'
)

![sch t sdny](plots / sch_sdny.png)

In [ ]:
plot_results.transport(
    results=results_cost, source='HO', sink='NY', resource='H2', transport='Train_H2'
)

![sch hony](plots / sch_hony.png)

**Cost contribution by technology**

In [ ]:
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.CAPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.VOPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.FOPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.TOTAL,
    location='HO',
    fig_size=(8, 6),
)

![total cost ho](plots / total_ho.png)

**Location-wise cost contribution by technology**

In [ ]:
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.CAPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.VOPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.FOPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.TOTAL, fig_size=(8, 6)
)

![total lw](plots / total_lw.png)

**Inventory and production schedules**

In [ ]:
plot_results.schedule(
    results=results_cost,
    y_axis='Inv',
    component='LiI_Power_stored',
    location='SD',
    fig_size=(9, 5),
    color='steelblue',
)

![inv lii](plots / lii_inv.png)

In [ ]:
plot_results.schedule(
    results=results_cost,
    y_axis='P',
    component='PV',
    location='SD',
    fig_size=(9, 5),
    color='steelblue',
)

![pv p](plots / pv_p.png)

## Optimize to maximize resource discharge 

In [ ]:
milp_demand = formulate(
    scenario=scenario,
    constraints={
        Constraints.COST,
        Constraints.INVENTORY,
        Constraints.PRODUCTION,
        Constraints.RESOURCE_BALANCE,
        Constraints.TRANSPORT,
        Constraints.MODE,
        Constraints.NETWORK,
    },
    objective=Objective.MAX_DISCHARGE,
    objective_resource=Power,
)

In [ ]:
milp_demand.constraint_specific_location_discharge_ho = (
    constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location=houston
    )
)
milp_demand.constraint_specific_location_discharge_sd = (
    constraint_specific_location_discharge(
        instance=milp_demand,
        network_scale_level=0,
        bounds={Power: 0},
        location=sandiego,
    )
)

In [ ]:
results_demand = solve(
    scenario=scenario,
    instance=milp_demand,
    solver='gurobi',
    name="results_demand",
    print_solversteps=True,
)

**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(
    results=results_demand, source='SD', sink='NY', resource='H2', transport='Train_H2'
)

![sch t sdny2](plots / sch_sdny2.png)

In [ ]:
plot_results.transport(
    results=results_demand, source='HO', sink='NY', resource='H2', transport='Train_H2'
)

![sch t hony](plots / sch_hony2.png)